In [1]:
import pandas as pd
import numpy as np
import os, sys

idx = pd.IndexSlice

In [2]:
data_fp = '../data/final-data.csv'
df = pd.read_csv(data_fp, index_col=None, header=[0, 1], skipinitialspace=True)

df.rename(columns={
    'Unnamed: 0_level_1': '',
    'Unnamed: 1_level_1': '',
    'Unnamed: 2_level_1': '',
    'Unnamed: 3_level_1': ''
}, level=1, inplace=True)
df['benchmark_id'] = df.index.values

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

model  log decomposition  SP label Cost of the alignment                    \
                                                     astar inc3 recomp-astar   
0  P430  L19   sese_manual  case_785                     0    0          0.0   
1  P430  L19   sese_manual   case_69                     0    0          0.0   
2  P430  L19   sese_manual   case_33                     0    0          0.0   
3  P430  L19   sese_manual  case_797                     4    4          4.0   
4  P430  L19   sese_manual  case_784                     0    0          0.0   

              Heuristics computed          ...       \
  recomp-inc3               astar inc3     ...        
0         0.0                   1    1     ...        
1         0.0                   1    1     ...        
2         0.0                   1    1     ...        
3         4.0                   1    1     ...        
4         0.0                   1    1     ...        

               model_trace_features                                     \
  n_uniq_tran_more_than_five_in_deg n_uniq_tran_more_than_five_out_deg   
0                                 0                                  0   
1                                 0                                  0   
2                                 0                                  0   
3                                 0                                  0   
4                                 0                                  0   

  Total Time including setup (us)                          \
                  astar_over_inc3 astar_over_recomp-astar   
0                        0.610397                0.639635   
1                        0.423914                0.473328   
2                        0.440726                0.360807   
3                        0.506494                0.659465   
4                        0.450129                0.349957   

                                                                       \
  astar_over_recomp-inc3 inc3_over_recomp-astar inc3_over_recomp-inc3   
0               0.505112               1.047901              0.827515   
1               0.350987               1.116568              0.827969   
2               0.268440               0.818665              0.609087   
3               0.499782               1.302021              0.986750   
4               0.276768               0.777459              0.614864   

                                          benchmark_id  
  recomp-astar_over_recomp-inc3  max_diff               
0                      0.789688  1.047901            0  
1                      0.741530  1.116568            1  
2                      0.744000  0.818665            2  
3                      0.757860  1.302021            3  
4                      0.790864  0.790864            4  

[5 rows x 263 columns]

In [4]:
df.shape

(245478, 263)

## Need to create the following csv files for LLAMA package
1. features.csv: features of the alignment problem
2. performance.csv: performance data of alignment computation
3. success.csv: whether if each algorithm could solve the alignment within timeout threshold

In [5]:
df.columns.get_level_values(level=0).unique()

Index(['model', 'log', 'decomposition', 'SP label', 'Cost of the alignment',
       'Heuristics computed', 'Heuristics derived', 'Heuristics estimated',
       'Markings added to closed set', 'Markings polled from queue',
       'Markings queued', 'Markings reached', 'Min',
       'Number of splits when splitting marking',
       'Number of times replay was restarted', 'Size of the constraintset',
       'Time to compute alignment (us)', 'Total Time including setup (us)',
       'is_valid', 'n_invis_move', 'n_log_move', 'n_model_move', 'n_sync_move',
       'result_dir', 'model_trace_features', 'benchmark_id'],
      dtype='object')

In [6]:
feature_df = df.loc[:, idx[('benchmark_id', 'model_trace_features'), :]]
feature_df.rename(columns={
    '': 'benchmark_id'
}, level=1, inplace=True)
feature_df.columns = list(feature_df.columns.get_level_values(level=1))

# filter unreported features
def get_feature_cols(columns):
    not_min_max = lambda c: not c.endswith('min') and not c.endswith('max')
    not_one_deg = lambda c: not c.endswith('one_in_deg') and not c.endswith('one_out_deg')
    not_two_deg = lambda c: not c.endswith('two_in_deg') and not c.endswith('two_out_deg')
    not_three_deg = lambda c: not c.endswith('three_in_deg') and not c.endswith('three_out_deg')
    not_more_than_five_deg = lambda c: not c.endswith('five_in_deg') and not c.endswith('five_out_deg')
    joined_filter = lambda c: not_min_max(c) \
                            and not_one_deg(c) \
                            and not_two_deg(c) \
                            and not_three_deg(c) \
                            and not_more_than_five_deg(c)
    return list(filter(joined_filter, columns))

feature_cols = get_feature_cols(list(feature_df.columns))
feature_df = feature_df.loc[:, feature_cols]
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', feature_df.columns))
feature_df = feature_df[reordered_cols]

# non_feature_colnames = df.columns.get_level_values(level=0)
# non_feature_colnames = list(filter(
#     lambda colname: colname != 'model_trace_features' \
#     and colname != 'model' \
#     and colname != 'log' \
#     and colname != 'decomposition' \
#     and colname != 'SP label' \
#     and colname != 'is_valid' \
#     and colname != 'result_dir'
# , non_feature_colnames))
performance_df = df.loc[:, idx[('benchmark_id', 'Total Time including setup (us)'), ('astar', 'inc3', 'recomp-astar', 'recomp-inc3', '')]]
performance_df.rename(columns={
#     'astar': 'astar_total_time_(us)',
#     'inc3': 'inc3_total_time_(us)',
#     'recomp-astar': 'recomp-astar_total_time_(us)',
#     'recomp-inc3': 'recomp-inc3_total_time_(us)',
    '': 'benchmark_id'
}, level=1, inplace=True)
performance_df.columns = list(performance_df.columns.get_level_values(level=1))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', performance_df.columns))
performance_df = performance_df[reordered_cols]

success_df = df.loc[:, idx[('benchmark_id', 'is_valid'), :]]
success_df.rename(columns={
    '': 'benchmark_id'
}, level=1, inplace=True)
success_df.columns = list(success_df.columns.get_level_values(level=1))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', success_df.columns))
success_df = success_df[reordered_cols]

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


## Export the dataframes

In [7]:
outdir = '../data/llama'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
feature_fp = os.path.join(outdir, 'features.csv')
performance_fp = os.path.join(outdir, 'performance.csv')
success_fp = os.path.join(outdir, 'success.csv')

feature_df.to_csv(feature_fp, index=False)
performance_df.to_csv(performance_fp, index=False)
success_df.to_csv(success_fp, index=False)